In [1]:
from ortools.linear_solver import pywraplp

In [88]:
#data
days = 5
slots_per_day = 4
nb_resources = 3
nb_projects = 4

squash_factor= 1

requested_slots_per_project = [4,5,3,4]

total_time_slots = days * slots_per_day
print(total_time_slots)

20


In [89]:
# Create the MIP solver
solver = pywraplp.Solver.CreateSolver('SCIP') # MIXED_INTEGER_PROGRAMMING

In [90]:
# variables definition

assignation = {}
for r in range(nb_resources):
    for p in range(nb_projects):
        for t in range(total_time_slots):
            assignation[(r, p, t)] = solver.BoolVar(f"assignation_r{r}_p{p}_t{t}")
zs = {}
zf = {}
for p in range(nb_projects):
    zs[(p)] = solver.IntVar(0, total_time_slots, f"zs_p{p}")
    zf[(p)] = solver.IntVar(0, total_time_slots, f"zf_p{p}")

y = solver.IntVar(0, total_time_slots, f"y")

In [91]:
print("Number of variables =", solver.NumVariables())

Number of variables = 249


In [92]:
# Define the objective function
objective = solver.Objective()
objective.SetCoefficient(y, 1)
objective.SetMinimization()


In [93]:
# constraints no double booking
for r in range(nb_resources):
    for t in range(total_time_slots):
        solver.Add(sum(assignation[(r, p, t)] for p in range(nb_projects)) <= 1)


In [94]:
# constraints
# Define constraint: Total working time for each project is equal to 10
for p in range(nb_projects):
    solver.Add(sum(assignation[(r, p, t)] for r in range(nb_resources) for t in range(total_time_slots)) == requested_slots_per_project[p])


In [95]:
for p in range(nb_projects):
    for t in range(total_time_slots):
        solver.Add(t * assignation[(r,p,t)] <= zf[p])

for p in range(nb_projects):
    for t in range(total_time_slots):
        solver.Add(zs[p] <= t*assignation[(r,p,t)]+(1-assignation[(r,p,t)])*99*total_time_slots)

# constraint assign lower bound of y
for r in range(nb_resources):
    for p in range(nb_projects):
        for t in range(total_time_slots):
            solver.Add(t * assignation[(r,p,t)] <= y)

for p in range(nb_projects):
    solver.Add(zf[p]-zs[p] <= requested_slots_per_project[p]*squash_factor)

In [96]:
print(solver.NumConstraints())

468


In [97]:
solver.Solve()

0

In [108]:
print('Objective value =', solver.Objective().Value())
print('p=0 zs:', zs[0].solution_value(), " zf:",zf[0].solution_value())
print('p=1 zs:', zs[1].solution_value(), " zf:",zf[1].solution_value())
print('p=2 zs:', zs[2].solution_value(), " zf:",zf[2].solution_value())
print('p=3 zs:', zs[3].solution_value(), " zf:",zf[3].solution_value())

print('z =', y.solution_value())

for r in range(nb_resources):
    for p in range(nb_projects):
        for t in range(total_time_slots):
            if(assignation[(r, p, t)].solution_value() == 1.0):
                print("resource:", r, " project: ", p, " time:",t)

#print(assignation[2,0,0].solution_value())



Objective value = 5.0
p=0 zs: 0.0  zf: 4.0
p=1 zs: 1.0  zf: 6.0
p=2 zs: 8.0  zf: 11.0
p=3 zs: 0.0  zf: 4.0
z = 5.0
resource: 0  project:  1  time: 0
resource: 0  project:  1  time: 1
resource: 0  project:  1  time: 2
resource: 0  project:  1  time: 3
resource: 0  project:  1  time: 4
resource: 0  project:  2  time: 5
resource: 1  project:  0  time: 4
resource: 1  project:  0  time: 5
resource: 1  project:  2  time: 0
resource: 1  project:  2  time: 1
resource: 1  project:  3  time: 2
resource: 1  project:  3  time: 3
resource: 2  project:  0  time: 0
resource: 2  project:  0  time: 1
resource: 2  project:  3  time: 2
resource: 2  project:  3  time: 3
